In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.graph_objects as go
from ipywidgets import widgets
import seaborn as sns
import folium
from folium.plugins import TimestampedGeoJson
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.


In [ ]:
#Reading Needed files
df_2016=pd.read_csv("/kaggle/input/air-pollution-dataset-india20162018/2016_india_withoutNA.csv")
df_2017=pd.read_csv("/kaggle/input/air-pollution-dataset-india20162018/2017_india_withoutNA.csv")
df_2018=pd.read_csv("/kaggle/input/air-pollution-dataset-india20162018/2018_india_withoutNA.csv")
df_temp=df_2016.append(df_2017,ignore_index=True)
df=df_temp.append(df_2018,ignore_index=True)

In [ ]:
#getting dates for diwali days
y16_oct = [20,21,22,23,24,25,26,27,28,29, 30 ,31]
y16_nov = ['01','02','03','04','05','06','07','08','09']
y17 = ['09',10,11,12,13,14,15,16,17,18, 19 ,20,21,22,23,24,25,26,27,28,29]

y_comb = [y16_oct, y16_nov, y17]
year_month = ["2016-10-", "2016-11-", "2017-10-"]
dat = []
for i in range(len(y_comb)):
    for j in range(len(y_comb[i])):
        y_comb[i][j] = str(y_comb[i][j])

print("Y comb = ", y_comb)

In [ ]:
#Dropdown for diwali, based on city and pollutant
#The value at x = 10 (middle of the graph) is the pollutant level on diwali
#I have considered values 10 days before and after to see the trend for that period

#need to add two hprizontal lines indicating the five day period or the actual diwali day ........!!!!!!.......
from ipywidgets import widgets
#-----------------------------------------creating temp dataframe--------------------------
df_sub = grouped_daily.loc[grouped_daily['city'] == 'Delhi']
df_sub = df_sub.loc[df_sub['parameter'] == 'co']

years_data = []
for ym in range(len(year_month)):
    y = []
    for d in y_comb[ym]:
        c = year_month[ym] + d
        df1_sub = df_sub[df_sub['date'].astype(str).str.contains(c)]
        yd = df1_sub.value.mean()
        y.append(yd)
    years_data.append(y)

y16 = []
for i in years_data[0]:
    y16.append(i)
for i in years_data[1]:
    y16.append(i)

y17_data = []
for i in years_data[2]:
    y17_data.append(i)
#------------------------------------------------------------------------------------------

city = widgets.Dropdown(
    description='City:   ',
    value='Delhi',
    options=grouped_daily['city'].unique().tolist()
)
parameter = widgets.Dropdown(
    options=list(grouped_daily['parameter'].unique()),
    value='co',
    description='Parameter:   ',
)

trace1 = go.Scatter(y=y16, mode='lines', name='year 2016')
trace2 = go.Scatter(y=y17_data, mode='lines', name='Year 2017')
g = go.FigureWidget(data=[trace1, trace2],
                    layout=go.Layout(
                        title=dict(
                            text='Pollutant levels during diwali days for 2016,2017'
                        )
                    ))

def response(change):
        #-----------------------------------------creating temp dataframe--------------------------
        df_sub = grouped_daily.loc[grouped_daily['city'] == city.value]
        df_sub = df_sub.loc[df_sub['parameter'] == parameter.value]

        years_data = []
        for ym in range(len(year_month)):
            y = []
            for d in y_comb[ym]:
                c = year_month[ym] + d
                df1_sub = df_sub[df_sub['date'].astype(str).str.contains(c)]
                yd = df1_sub.value.mean()
                y.append(yd)
            years_data.append(y)

        y16 = []
        for i in years_data[0]:
            y16.append(i)
        for i in years_data[1]:
            y16.append(i)

        y17_data = []
        for i in years_data[2]:
            y17_data.append(i)        
        #------------------------------------------------------------------------------------------
        y0 = y16
        y1 = y17_data
        with g.batch_update():
            g.data[0].y = y0
            g.data[1].y = y1
            g.layout.xaxis.title = 'Date'
            g.layout.yaxis.title = 'Pollutant levels'


city.observe(response, names="value")
parameter.observe(response, names="value")
container2 = widgets.HBox([city, parameter])
widgets.VBox([container2,g])

In [ ]:
#function to get the day from the given date
from datetime import date
import calendar

def get_day(my_date):
    #adding new line
    my_date = str(my_date)
    y = int(my_date[:4])
    #print(y)
    m = int(my_date[5:7])
    #print(m)
    d = int(my_date[8:10])
    #print(d)
    my_day = date(y, m, d).weekday()
    return calendar.day_name[my_day]

In [ ]:
#dropdown for weekend, weekday
#select pollutant, city
from ipywidgets import widgets

#-----------------------------------------creating temp dataframe--------------------------
weekday = pd.DataFrame()
weekend = pd.DataFrame()
df_no2 = grouped_daily.loc[grouped_daily['parameter'] == 'no2']
df_no2 = df_no2.loc[df_no2['city'] == 'Delhi']

for i in range(len(df_no2)):
    if ((get_day(df_no2.iloc[i,3]) == 'Sunday') or (get_day(df_no2.iloc[i,3]) == 'Saturday')):
        weekend = weekend.append(df_no2.iloc[i,:])
    else:
        weekday = weekday.append(df_no2.iloc[i,:])
#------------------------------------------------------------------------------------------

city = widgets.Dropdown(
    description='City:   ',
    value='Delhi',
    options=grouped_daily['city'].unique().tolist()
)
parameter = widgets.Dropdown(
    options=list(grouped_daily['parameter'].unique()),
    value='co',
    description='Parameter:   ',
)

trace1 = go.Scatter(x=weekend['date'], y=weekend['value'], mode='markers', name='Weekends')
trace2 = go.Scatter(x=weekday['date'], y=weekday['value'], mode='markers', name='Weekdays')
g = go.FigureWidget(data=[trace1, trace2],
                    layout=go.Layout(
                        title=dict(
                            text='Pollutant levels on weekends vs weekdays'
                        )
                    ))
def response(change):

        #-----------------------------------------creating temp dataframe--------------------------
        weekday = pd.DataFrame()
        weekend = pd.DataFrame()
        df_no2 = grouped_daily.loc[grouped_daily['parameter'] == parameter.value]
        df_no2 = df_no2.loc[df_no2['city'] == city.value]

        for i in range(len(df_no2)):
            if ((get_day(df_no2.iloc[i,3]) == 'Sunday') or (get_day(df_no2.iloc[i,3]) == 'Saturday')):
                weekend = weekend.append(df_no2.iloc[i,:])
            else:
                weekday = weekday.append(df_no2.iloc[i,:])
        #------------------------------------------------------------------------------------------

        filter_list = [i and j for i, j in
        zip(grouped_daily['city'] == city.value, grouped_daily['parameter'] == parameter.value)]
        temp_df = grouped_daily[filter_list]
        x0 = weekend['date']
        y0 = weekend['value']
        x1 = weekday['date']
        y1 = weekday['value']
        with g.batch_update():
            g.data[0].x = x0
            g.data[0].y = y0
            g.data[1].x = x1
            g.data[1].y = y1
            g.layout.xaxis.title = 'Date'
            g.layout.yaxis.title = 'Pollutant levels'


city.observe(response, names="value")
parameter.observe(response, names="value")
container2 = widgets.HBox([city, parameter])
widgets.VBox([container2,g])

In [ ]:
#weekend weekday for a particular year (2016), along with boxplot.
#The points are not separated into distinct bands of weekday(wider), weekend(narrower band) bcoz very less points
#one point for a day

weekday = pd.DataFrame()
weekend = pd.DataFrame()

df_no2 = grouped_daily.loc[grouped_daily['parameter'] == 'no2']
df_no2 = df_no2.loc[df_no2['city'] == 'Delhi']
c = "2016"
df_no2 = df_no2[df_no2['date'].astype(str).str.contains(c)]

df_no2

for i in range(len(df_no2)):
    if ((get_day(df_no2.iloc[i,3]) == 'Sunday') or (get_day(df_no2.iloc[i,3]) == 'Saturday')):
        #print("hello")
        weekend = weekend.append(df_no2.iloc[i,:])
    else:
        weekday = weekday.append(df_no2.iloc[i,:])
print("len = ", len(df_no2))

#plotting weekend vs weekday for no2, delhi
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=weekend['date'], y=weekend['value'], mode='markers', name='Weekends'))
fig.add_trace(go.Scatter(x=weekday['date'], y=weekday['value'], mode='markers', name='Weekdays'))
fig.show()

#boxplot
fig = plt.figure()
ax = fig.add_subplot(111)
ax.boxplot([weekend['value'],weekday['value']], labels=['Weekend', 'Weekday'])
ax.set_ylabel('Concentration on NO2')
plt.show()

In [ ]:
#splitting the data into weekend, weekday for a particular location, without grouping
#has much more distinct bands for weekend weekdays and the trend is shown more clearly.
#coz there are more points

weekday = pd.DataFrame()
weekend = pd.DataFrame()

df_no2 = df.loc[df['parameter'] == 'no2']
df_no2 = df_no2.loc[df_no2['city'] == 'Delhi']
df_no2 = df_no2.loc[df_no2['location'] == 'RK Puram']

for i in range(len(df_no2)):
    if ((get_day(df_no2.iloc[i,4]) == 'Sunday') or (get_day(df_no2.iloc[i,4]) == 'Saturday')):
        #print("hello")
        weekend = weekend.append(df_no2.iloc[i,:])
    else:
        weekday = weekday.append(df_no2.iloc[i,:])

print("len = ", len(df_no2))

#plotting weekend vs weekday for no2, RK puram, all days., without grouping
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=weekend['local'], y=weekend['value'], mode='markers', name='Weekends'))
fig.add_trace(go.Scatter(x=weekday['local'], y=weekday['value'], mode='markers', name='Weekdays'))
fig.show()

#Boxplot
fig = plt.figure()
ax = fig.add_subplot(111)
ax.boxplot([weekend['value'],weekday['value']], labels=['Weekend', 'Weekday'])
ax.set_ylabel('Concentration on NO2')
plt.show()